In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from datetime import date
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
import re

In [47]:
df1 = pd.read_parquet("df_final_compile.parquet")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df1.head(3)

,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine transforme les haillons de Cendrillon en robe féérique et, dans une citrouille changée en carrosse, l’envoie au bal. Mais le charme se rompt à minuit. Les souris, Jaq et Gus, et une pantoufle de vair contribueront au dénouement d'un conte de fées des plus mémorables.",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1903-10-16,1968-02-19,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,"Un jeune homme d'origine modeste est accusé du meurtre de son père et risque la peine de mort. Le jury composé de douze hommes se retire pour délibérer et procède immédiatement à un vote : onze votent coupable, or la décision doit être prise à l'unanimité. Le juré qui a voté non-coupable, sommé de se justifier, explique qu'il a un doute et que la vie d'un homme mérite quelques heures de discussion. Il s'emploie alors à les convaincre un par un.",58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1924-06-25,2011-04-09,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,"Évocation de la vie de Moïse sauvé à sa naissance par la propre fille du pharaon qui avait décidé de mettre à mort tous les nouveaux-nés hébreux de peur que les fils d'Israël ne deviennent plus nombreux que les Égyptiens. Cecil B. de Mille avait déjà tourné une version des Dix Commandements en 1923. Il récidive trente ans plus tard pour une version démesurée, sept mois de tournage, cent mille accessoires, des chameaux venus d'Australie, des lotus cueillis en Guinée britannique...",56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1881-08-12,1959-01-21,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01


In [38]:
df1.shape

(5771, 38)

In [4]:
df_movie = df1.copy()

In [5]:
# supprimer les colonnes non utilisées dans ML
df_movie = df_movie.drop(columns=['backdrop_path', 'overview', 'poster_path', 'movie_id', 'actor_birthday_1', 'actor_birthday_2',
       'actor_birthday_3', 'actor_birthday_4', 'actor_birthday_5'])

In [7]:
date_list = ['release_date', 'birthday_director', 'deathday_director']

for date in date_list:
    df_movie[date] = pd.to_datetime(df_movie[date])

df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date', 'birthday_director', 'deathday_director']].fillna(0)

for date in date_list:
    df_movie[date] = ((df_movie[date].dt.year // 10) * 10).astype(int)

df_movie

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5
0,11224,en,79.005,1970,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626
1,389,en,58.940,1970,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698
2,6844,en,56.194,1970,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539
3,665,en,56.369,1970,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1900,1980,"Mülhausen, Alsace-Lorraine, German Empire [now...",Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,2.0,2.0,2.0,2.0,1.0,22.823,8.798,4.333,9.360,2.899
4,10882,en,46.057,1970,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1900,1980,"Chiavenna, Lombardy, Italy",Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,1.0,2.0,1.0,1.0,1.0,4.002,2.951,7.009,8.036,3.495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,694234,it,7.660,1970,Sous le soleil de Riccione,6.200,552,"[Comedy, Romance]",2,Antonio Usbergo,0.335,1970,1970,None,Cristiano Caccamo,Saul Nanni,Andrea Roncato,Lorenzo Zurzolo,Ludovica Martino,2.0,2.0,2.0,2.0,1.0,6.585,4.948,1.634,28.534,4.751
5767,714888,es,10.286,1970,"Argentine, 1985",7.900,617,"[Drama, History, Crime]",2,Santiago Mitre,0.674,1980,1970,"Buenos Aires, Argentina",Ricardo Darín,Peter Lanzani,Alejandra Flechner,Paula Ransenberg,Carlos Portaluppi,2.0,2.0,1.0,1.0,2.0,5.877,3.067,1.376,0.346,1.270
5768,606954,it,6.531,1970,Odio l'estate,7.100,921,[Comedy],2,Massimo Venier,3.630,1960,1970,"Varese, Italy",Aldo Baglio,Giovanni Storti,Giacomo Poretti,Lucia Mascino,Carlotta Natoli,2.0,2.0,2.0,1.0,1.0,3.680,2.184,1.579,5.234,2.985
5769,614292,it,6.972,1970,Storia di vacanze,6.742,567,[Drama],2,Damiano D'Innocenzo,1.030,1980,1970,"Rome, Lazio, Italy",Elio Germano,Tommaso Di Cola,Giulietta Rebeggiani,Gabriel Montesi,Justin Alexander Korovkin,2.0,2.0,1.0,2.0,2.0,4.840,0.662,1.511,1.177,0.636


In [43]:
df_movie.columns

Index(['original_language', 'popularity_movie', 'release_date', 'vote_average',
       'vote_count', 'genre', 'gender_director', 'name_director',
       'popularity_director', 'birthday_director', 'deathday_director',
       'place_of_birth_director', 'actor_name_1', 'actor_name_2',
       'actor_name_3', 'actor_name_4', 'actor_name_5', 'actor_gender_1',
       'actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
       'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5'],
      dtype='object')

In [8]:
actor_list = []
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    for i in range(0, len(df_movie)): 
        acteur_name = df_movie.loc[i, acteur_name_col]
        if acteur_name not in actor_list: 
            actor_list.append(acteur_name)

In [9]:
resultat = {}

for acteur in actor_list: 
    index_list = []
    popularity_list = []
    for col in range(1, 6): 
        acteur_name_col = f"actor_name_{col}"
        
        index = df_movie[df_movie[acteur_name_col] == acteur].index
        for i in index : 
            index_list.append(i)

    for index in index_list:
        popularity = df_movie.loc[index, 'popularity_movie']
        popularity_list.append(popularity)
    
    try: 
        resultat[acteur] = float(round(sum(popularity_list) / len(popularity_list),2))
    except: 
        pass


In [10]:
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    for i in range(0, len(df_movie)): 
        df_movie.loc[i, acteur_name_col] = resultat.get(df_movie.loc[i, acteur_name_col])

df_movie.head(3)


,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5
0,11224,en,79.005,1970,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",79.0,62.53,62.53,79.0,79.0,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626
1,389,en,58.940,1970,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",27.72,27.4,44.92,58.94,58.94,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698
2,6844,en,56.194,1970,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",29.45,38.47,56.19,56.19,56.19,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539


In [11]:
df_movie = df_movie.rename(columns={'actor_name_1': 'actor_1_mean_movie', 
                         'actor_name_2': 'actor_2_mean_movie', 
                         'actor_name_3': 'actor_3_mean_movie',
                         'actor_name_4': 'actor_4_mean_movie', 
                         'actor_name_5': 'actor_5_mean_movie'
})

In [76]:
df_movie.head(3)

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_1_mean_movie,actor_2_mean_movie,actor_3_mean_movie,actor_4_mean_movie,actor_5_mean_movie,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5
0,11224,en,79.005,1970,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",79.0,62.53,62.53,79.0,79.0,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626
1,389,en,58.940,1970,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",27.72,27.4,44.92,58.94,58.94,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698
2,6844,en,56.194,1970,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",29.45,38.47,56.19,56.19,56.19,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539


In [77]:
df_movie.isnull().sum()

id_movie                     0
original_language            0
popularity_movie             0
release_date                 0
title                        0
vote_average                 0
vote_count                   0
genre                        0
gender_director              0
name_director                0
popularity_director          0
birthday_director            0
deathday_director            0
place_of_birth_director    435
actor_1_mean_movie           0
actor_2_mean_movie           3
actor_3_mean_movie          10
actor_4_mean_movie          33
actor_5_mean_movie         242
actor_gender_1               0
actor_gender_2               3
actor_gender_3              10
actor_gender_4              33
actor_gender_5             242
actor_popularity_1           0
actor_popularity_2           3
actor_popularity_3          10
actor_popularity_4          33
actor_popularity_5         242
dtype: int64

In [12]:
# Remplir les valeurs nulles dans d'autres colonnes numériques
df_movie[['actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
          'actor_2_mean_movie', 'actor_3_mean_movie', 'actor_4_mean_movie', 'actor_5_mean_movie', 
          'actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4', 'actor_popularity_5']] = df_movie[['actor_gender_2', 'actor_gender_3', 
                                                                                                               'actor_gender_4', 'actor_gender_5',
                                                                                                               'actor_2_mean_movie', 'actor_3_mean_movie', 
                                                                                                               'actor_4_mean_movie', 'actor_5_mean_movie', 
                                                                                                               'actor_popularity_2', 'actor_popularity_3', 
                                                                                                               'actor_popularity_4', 'actor_popularity_5']].fillna(0)



# Remplir les valeurs nulls dans d'autres colonnes non-numériques : 
df_movie[['place_of_birth_director']] = df_movie[['place_of_birth_director']].fillna('aucun')



C:\Users\sulta\AppData\Local\Temp\ipykernel_22944\289357162.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'actor_popularity_4', 'actor_popularity_5']].fillna(0)


In [13]:
df_movie.isnull().sum()

id_movie                   0
original_language          0
popularity_movie           0
release_date               0
title                      0
vote_average               0
vote_count                 0
genre                      0
gender_director            0
name_director              0
popularity_director        0
birthday_director          0
deathday_director          0
place_of_birth_director    0
actor_1_mean_movie         0
actor_2_mean_movie         0
actor_3_mean_movie         0
actor_4_mean_movie         0
actor_5_mean_movie         0
actor_gender_1             0
actor_gender_2             0
actor_gender_3             0
actor_gender_4             0
actor_gender_5             0
actor_popularity_1         0
actor_popularity_2         0
actor_popularity_3         0
actor_popularity_4         0
actor_popularity_5         0
dtype: int64

In [65]:
df_movie.columns

Index(['original_language', 'popularity_movie', 'release_date', 'title',
       'vote_average', 'vote_count', 'genre', 'gender_director',
       'name_director', 'popularity_director', 'birthday_director',
       'deathday_director', 'place_of_birth_director', 'actor_1_mean_movie',
       'actor_2_mean_movie', 'actor_3_mean_movie', 'actor_4_mean_movie',
       'actor_5_mean_movie', 'actor_gender_1', 'actor_gender_2',
       'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
       'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5'],
      dtype='object')

In [80]:
df_movie.shape

(5771, 29)

In [81]:
df_movie.to_parquet("df_movie.parquet")

ne pas garder title : ok, supprimé
name_director : ok
rajouter id_directeur : on n'a pas son id 

In [37]:
numeric_cols = ['popularity_movie', 'vote_average', 'vote_count',
       'gender_director', 'popularity_director', 'birthday_director', 'actor_2_mean_movie', 'actor_3_mean_movie',
       'actor_4_mean_movie', 'actor_5_mean_movie', 'actor_gender_1',
       'actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
       'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5']


categorical_cols = ['original_language', 'name_director', 'place_of_birth_director']

binarizer_cols = ['genre']

date_cols = ['release_date']

In [38]:
# Custom transformer for MultiLabelBinarizer
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

In [39]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

date_transformer = Pipeline(steps=[
    ('date', OrdinalEncoder())
])

# On combine tout dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genre'),
        ('date', date_transformer, date_cols)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [40]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['popularity_movie',
                                                   'vote_average', 'vote_count',
                                                   'gender_director',
                                                   'popularity_director',
                                                   'birthday_director',
                                                   'actor_2_mean_movie',
                                                   'actor_3_mean_movie',
                                                   'actor_4_mean_movie',
                                                   'actor_5_mean_movie',
                                                   'actor_gender_1',
                                                   'actor_gender_2',
                                                   'actor_...
                                                   'actor_popularity_2',
                                                   'actor_popularity_3',
                                                   'actor_popularity_4',
                                                   'actor_popularity_5']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['original_language',
                                                   'name_director',
                                                   'place_of_birth_director']),
                                                 ('genres',
                                                  MultiLabelBinarizerPipelineFriendly(),
                                                  'genre'),
                                                 ('date',
                                                  Pipeline(steps=[('date',
                                                                   OrdinalEncoder())]),
                                                  ['release_date'])]))])

In [41]:
# Transformation des données
processed_data = pipeline.fit_transform(df_movie)

# Affichage des données transformées
transformed_df = pd.DataFrame(processed_data.todense(), columns=pipeline["preprocessor"].get_feature_names_out())

In [19]:
processed_data.todense().shape

(5771, 3863)

In [42]:
# Initialisation du modèle KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='manhattan')  # 5 voisins les plus proches hormis le point de référence lui meme
knn_model.fit(processed_data)

NearestNeighbors(metric='manhattan', n_neighbors=6)

In [27]:
distances, indices = knn_model.kneighbors(processed_data)

c:\Users\sulta\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\sulta\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [28]:
distances[5]

array([ 0.        ,  6.24963777,  6.843566  ,  9.07348843, 11.59344421,
       12.60546004])

In [29]:
indices[2]

array([  2,  30,  23,   3, 153,  25])

In [43]:
# Exemple de recommandation pour un item
def recommend_similar_films(title, 
                              data: pd.DataFrame=processed_data, 
                              model: NearestNeighbors=knn_model, 
                              original_data: pd.DataFrame=df1, 
                              n_neighbors: int=6) -> tuple[list[float], list[float]]:
    """
    Trouve les n éléments les plus proches pour un élément donné.

    Args:
        title: le titre d'un film
        data: Données transformées utilisées pour KNN = df processed data
        model: Modèle KNN pré-entraîné.
        original_data: Données originales (pour affichage) = df movies
        n_neighbors: Nombre de voisins à recommander = 6 pour avoir 5 recommandations

    Returns:
        films similaires: Indices et distances des filmes similaires.
    """
    index = original_data[original_data['title'] == title].index[0] # l'indice du filme recherché 
    distances, indices = model.kneighbors(data[index], n_neighbors=n_neighbors) 
    print(f"Le filme recherché : {title}:\n")
    display(pd.DataFrame(original_data.iloc[index]).T)
    display(distances[0])
    print(f"Recommandations pour le film : {title} :\n")
    display(original_data.iloc[indices[0][1:]])


In [45]:
recommend_similar_films("Maman, j’ai encore raté l’avion !")

Le filme recherché : Maman, j’ai encore raté l’avion !:



,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5
787,772,/kjby7cr8JnIPkfqMGKutOFrM2at.jpg,en,"Après Paris, c'est à Miami que la tribu McCall...",160.071,/gFhf8NwvEKCbNIOQZxLqj1i5DMy.jpg,1992-11-15,"Maman, j’ai encore raté l’avion !",6.7,9522,"[Comedy, Family, Adventure]",2,Chris Columbus,9.968,1958-09-10,None,"Spangler, Pennsylvania, USA",772,Macaulay Culkin,Daniel Stern,Joe Pesci,Catherine O'Hara,John Heard,2.0,2.0,2.0,1.0,2.0,32.703,23.703,17.815,23.608,16.584,1980-08-26,1957-08-28,1943-02-09,1954-03-04,1945-03-07


array([ 0.        ,  7.19921857, 10.88724229, 12.16591095, 13.67755596,
       13.9362407 ])

Recommandations pour le film : Maman, j’ai encore raté l’avion ! :



,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5
1453,771,/xKG9aVg4GvNn9puHjUPaTNkNDcC.jpg,en,La famille McCallister a décidé de passer les ...,9.074,/aP9UrDpKS5i5PCiYKDgJvkRz7ne.jpg,1990-11-16,"Maman, j'ai raté l'avion !",7.400,11180,"[Comedy, Family]",2,Chris Columbus,9.968,1958-09-10,None,"Spangler, Pennsylvania, USA",771,Macaulay Culkin,Joe Pesci,Daniel Stern,Catherine O'Hara,John Heard,2.0,2.0,2.0,1.0,2.0,32.703,17.815,23.703,23.608,16.584,1980-08-26,1943-02-09,1957-08-28,1954-03-04,1945-03-07
1554,953,/tPaurpIUskVji5vwV0dhy9pq4Vs.jpg,en,"Lorsqu’un lion, un zèbre, une girafe, un hippo...",83.482,/aA1eZtISZ2rqUk2QIUgu1pXcHrm.jpg,2005-05-25,Madagascar,6.900,10800,"[Family, Animation, Adventure, Comedy]",2,Tom McGrath,8.131,1964-08-07,None,"Lynnwood, Washington, USA",953,Ben Stiller,Chris Rock,David Schwimmer,Jada Pinkett Smith,Sacha Baron Cohen,2.0,2.0,2.0,1.0,2.0,39.054,12.157,29.360,26.820,19.474,1965-11-30,1965-02-07,1966-11-02,1971-09-18,1971-10-13
1582,10527,/3yXLkSTrVhG66MIAr3X6C4gjccw.jpg,en,Après s’être échoué sur les rivages lointains ...,56.352,/eLCEqcUYVjAyFHDmuteNPja5YuO.jpg,2008-10-30,Madagascar 2,6.511,6841,"[Family, Adventure, Animation, Comedy]",2,Eric Darnell,3.007,1961-08-21,None,"Prairie Village, Kansas, USA",10527,Ben Stiller,Chris Rock,David Schwimmer,Jada Pinkett Smith,Sacha Baron Cohen,2.0,2.0,2.0,1.0,2.0,39.054,12.157,29.360,26.820,19.474,1965-11-30,1965-02-07,1966-11-02,1971-09-18,1971-10-13
818,9279,/rnn6sGMGPpM8VfVmCMFxsjKvz7q.jpg,en,"Comme tous les enfants américains, et bientôt ...",55.686,/pN98TAu71SfSVLPAUmWWDlKnTQx.jpg,1996-11-21,La Course au jouet,6.037,2600,"[Family, Comedy, Adventure]",2,Brian Levant,6.516,1952-08-06,None,"Highland Park, Illinois, USA",9279,Arnold Schwarzenegger,Sinbad,Phil Hartman,Rita Wilson,Robert Conrad,2.0,2.0,2.0,1.0,2.0,55.505,14.939,10.547,19.893,10.892,1947-07-30,1956-11-10,1948-09-24,1956-10-26,1935-03-01
2952,80321,/pGa4217asjzjepco86110xLWjE4.jpg,en,"Alex le lion, Marty le zèbre, Gloria l’hippopo...",66.242,/trMyHbr9CfcNYWcNERp8RorfmoW.jpg,2012-06-06,Madagascar 3 : Bons baisers d'Europe,6.600,6093,"[Animation, Family, Comedy, Adventure]",2,Conrad Vernon,6.122,1968-07-11,None,"Lubbock, Texas, USA",80321,Ben Stiller,Chris Rock,David Schwimmer,Jada Pinkett Smith,Frances McDormand,2.0,2.0,2.0,1.0,1.0,39.054,12.157,29.360,26.820,9.814,1965-11-30,1965-02-07,1966-11-02,1971-09-18,1957-06-23


In [46]:
df1['title'].sample(10)

3131                              Le Chemin du pardon
2365              Windtalkers : Les Messagers du vent
5412                                         Hamilton
2560                                    I'm Not There
2072    La Véritable Histoire du Petit Chaperon rouge
2731                                Carnets de voyage
3632                                   Phantom Thread
997                                Piège en haute mer
4102                                            Greta
749                                    Turner & Hooch
Name: title, dtype: object

In [33]:
df1.loc[1528, 'title']

'Pirates des Caraïbes\xa0: Le Secret du coffre maudit'